In [1]:
# Import the packages
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

In [2]:
# Download the data
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt

--2019-10-26 19:02:27--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
R'esolution de storage.googleapis.com (storage.googleapis.com)... 172.217.19.208
Connexion `a storage.googleapis.com (storage.googleapis.com)|172.217.19.208|:443... connect'e.
requ^ete HTTP transmise, en attente de la r'eponse... 200 OK
Taille : 93578 (91K) [text/plain]
Sauvegarde en : << /tmp/sonnets.txt >>

/tmp/sonnets.txt    100%[===================>]  91.38K   589KB/s    ds 0.2s    

2019-10-26 19:02:28 (589 KB/s) - << /tmp/sonnets.txt >> sauvegard'e [93578/93578]



In [3]:
# Transform data in a corpus
data = open('/tmp/sonnets.txt').read()
corpus = data.lower().split("\n")

In [4]:
# Tokenize the corpus
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [5]:
# Create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
# Pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, 
                                         maxlen = max_sequence_len, 
                                         padding = 'pre'))

In [7]:
# Create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes = total_words)

In [8]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len - 1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))
model.add(Dense(total_words, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
______________________________________________

In [10]:
# Training
history = model.fit(predictors, label, epochs = 10, verbose = 1)

Train on 15462 samples
Epoch 1/10
15462/15462 [==============================] - 99s 6ms/sample - loss: 6.4635 - accuracy: 0.0237
Epoch 2/10
15462/15462 [==============================] - 90s 6ms/sample - loss: 6.3414 - accuracy: 0.0293
Epoch 3/10
15462/15462 [==============================] - 90s 6ms/sample - loss: 6.2356 - accuracy: 0.0338
Epoch 4/10
15462/15462 [==============================] - 91s 6ms/sample - loss: 6.1535 - accuracy: 0.0384
Epoch 5/10
15462/15462 [==============================] - 90s 6ms/sample - loss: 6.0736 - accuracy: 0.0398
Epoch 6/10
15462/15462 [==============================] - 91s 6ms/sample - loss: 5.9982 - accuracy: 0.0426
Epoch 7/10
15462/15462 [==============================] - 90s 6ms/sample - loss: 5.9115 - accuracy: 0.0460
Epoch 8/10
15462/15462 [==============================] - 84s 5ms/sample - loss: 5.8203 - accuracy: 0.0510
Epoch 9/10
15462/15462 [==============================] - 81s 5ms/sample - loss: 5.7097 - accuracy: 0.0581
Epoch 10/10
15

In [11]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [12]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

Help me Obi Wan Kenobi, you're my only hope and be love in love love in my love and love and love in love and love in love in thee in thee in love be love in thee in thee in love i art love in love i art be love i be be and be and be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be and be be
